In [1]:
import numpy as np
from utils import build_dataset
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.MY_GNN import collate_molgraphs, EarlyStopping, run_a_train_epoch_heterogeneous, \
    run_an_eval_epoch_heterogeneous, set_random_seed, MGA,pos_weight,multi_weight_six,multi_weight_four
import os
import time
import pandas as pd
start = time.time()

# fix parameters of model
args = {}
args['device'] = "cuda" if torch.cuda.is_available() else "cpu"
args['atom_data_field'] = 'atom'
args['bond_data_field'] = 'etype'
args['descriptor_dim'] = 2133
args['descriptor'] = 2133
args['fpn_out'] = 2133
args['fp_2_dim'] = 512
args['dropout'] = 0.2
args['hidden_size'] = 256


args['classification_metric_name'] = 'roc_auc'
args['regression_metric_name'] = 'r2'
# model parameter
args['num_epochs'] = 300
args['patience'] = 50
args['batch_size'] = 128
args['mode'] = 'higher'
args['in_feats'] = 40
args['rgcn_hidden_feats'] = [64, 64]
args['classifier_hidden_feats'] = 320
args['rgcn_drop_out'] = 0.2
args['drop_out'] = 0.2
args['lr'] = 3
args['weight_decay'] = 5
args['loop'] = True

# task name (model name)
args['task_name'] = 'Toxicity'  # change
args['data_name'] = 'Toxicity'  # change
args['times'] = 5

# selected task, generate select task index, task class, and classification_num
args['select_task_list'] = ['toxicity_nontoxicity','toxicity_type_class','neurotoxicity_type_class','HC50']  # change

args['select_task_index'] = []
args['classification_num'] = 0
args['regression_num'] = 0
args['all_task_list'] = ['toxicity_nontoxicity','toxicity_type_class','neurotoxicity_type_class','HC50']   # change

# generate select task index
for index, task in enumerate(args['all_task_list']):
    if task in args['select_task_list']:
        args['select_task_index'].append(index)
# generate classification_num

classification_tasks = []
regression_tasks = []
for task in args['select_task_list']:
    if task in ['toxicity_nontoxicity','toxicity_type_class','neurotoxicity_type_class']:
        args['classification_num'] = args['classification_num'] + 1
        classification_tasks.append(task)  # 存储分类任务名
    if task in [ 'HC50'] :
        args['regression_num'] = args['regression_num'] + 1
        regression_tasks.append(task)  # 存储回归任务名
        
args['classification_tasks'] = classification_tasks
args['regression_tasks'] = regression_tasks

#print("Classification tasks:", args['classification_tasks'])
#print("Regression tasks:", args['regression_tasks'])

# generate classification_num
if args['classification_num'] != 0 and args['regression_num'] != 0:
    args['task_class'] = 'classification_regression'
if args['classification_num'] != 0 and args['regression_num'] == 0:
    args['task_class'] = 'classification'
if args['classification_num'] == 0 and args['regression_num'] != 0:
    args['task_class'] = 'regression'
print('Classification task:{}, Regression Task:{}'.format(args['classification_num'], args['regression_num']))
args['bin_path'] = 'data/' + args['data_name'] + '.bin'
args['group_path'] = 'data/' + args['data_name'] + '_group.csv'


result_pd = pd.DataFrame(columns=args['select_task_list']+['group'] + args['select_task_list']+['group']
                         + args['select_task_list']+['group'])
all_times_train_result = []
all_times_val_result = []
all_times_test_result = []
for time_id in range(args['times']):
    set_random_seed(2020+time_id)
    one_time_train_result = []
    one_time_val_result = []
    one_time_test_result = []
    print('***************************************************************************************************')
    print('{}, {}/{} time'.format(args['task_name'], time_id+1, args['times']))
    print('***************************************************************************************************')
    train_set, val_set, test_set, task_number = build_dataset.load_graph_from_csv_bin_for_splited(
        bin_path=args['bin_path'],
        group_path=args['group_path'],
        select_task_index=args['select_task_index']
    )
    print("Molecule graph generation is complete !")
    train_loader = DataLoader(dataset=train_set,
                              batch_size=args['batch_size'],
                              shuffle=True,
                              collate_fn=collate_molgraphs)

    val_loader = DataLoader(dataset=val_set,
                            batch_size=args['batch_size'],
                            shuffle=True,
                            collate_fn=collate_molgraphs)

    test_loader = DataLoader(dataset=test_set,
                             batch_size=args['batch_size'],
                             collate_fn=collate_molgraphs)  

    pos_weight_np = pos_weight(train_set)
    weight1 = multi_weight_six(train_set)
  #  print(f"weight1: {weight1}")

    weight2 = multi_weight_four(train_set)
 #   print(f"weight2: {weight2}")

    loss_criterion_c_0 = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight_np.to(args['device']))
    loss_criterion_c_1 = torch.nn.CrossEntropyLoss(reduction='none',weight=torch.tensor(weight1).to(args['device']))
    loss_criterion_c_2 = torch.nn.CrossEntropyLoss(reduction='none',weight=torch.tensor(weight2).to(args['device'])) 
    loss_criterion_r = torch.nn.MSELoss(reduction='none')
    
    model = MGA(in_feats=args['in_feats'],descriptor=args['descriptor'],descriptor_dim=args['descriptor_dim'],rgcn_hidden_feats=args['rgcn_hidden_feats'],
                n_tasks=task_number, rgcn_drop_out=args['rgcn_drop_out'],fpn_out=args['fpn_out'],fp_2_dim=args['fp_2_dim'],
                hidden_size=args['hidden_size'],select_task_list=args['select_task_list'],device=args['device'],
                classifier_hidden_feats=args['classifier_hidden_feats'], dropout=args['drop_out'],
                loop=args['loop'])
    optimizer = Adam(model.parameters(), lr=10**-args['lr'], weight_decay=10**-args['weight_decay'])
    stopper = EarlyStopping(patience=args['patience'], task_name=args['task_name'], mode=args['mode'])
    model.to(args['device'])

    for epoch in range(args['num_epochs']):
        # Train
        run_a_train_epoch_heterogeneous(args, epoch, model, train_loader, loss_criterion_c_0,loss_criterion_c_1,loss_criterion_c_2,
                                        loss_criterion_r, optimizer)

        # Validation and early stop
        validation_result = run_an_eval_epoch_heterogeneous(args, model, val_loader)
        val_score = np.mean(validation_result)
        early_stop = stopper.step(val_score, model)
        print('epoch {:d}/{:d}, validation {:.4f}, best validation {:.4f}'.format(
            epoch + 1, args['num_epochs'],
            val_score,  stopper.best_score)+' validation result:', validation_result)
        if early_stop:
            break
    stopper.load_checkpoint(model)
    test_score = run_an_eval_epoch_heterogeneous(args, model, test_loader)
    train_score = run_an_eval_epoch_heterogeneous(args, model, train_loader)
    val_score = run_an_eval_epoch_heterogeneous(args, model, val_loader)
    # deal result
    result = train_score + ['training'] + val_score + ['valid'] + test_score + ['test']
    result_pd.loc[time_id] = result
    print('********************************{}, {}_times_result*******************************'.format(args['task_name'], time_id+1))
    print("training_result:", train_score)
    print("val_result:", val_score)
    print("test_result:", test_score)

result_pd.to_csv('result/'+args['task_name']+'_result.csv', index=None)
elapsed = (time.time() - start)
m, s = divmod(elapsed, 60)
h, m = divmod(m, 60)
print("Time used:", "{:d}:{:d}:{:d}".format(int(h), int(m), int(s)))

C:\Users\14827\anaconda3\envs\my-rdkit-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classification task:3, Regression Task:1
***************************************************************************************************
Toxicity, 1/5 time
***************************************************************************************************
21192 2648 2656 4
Molecule graph generation is complete !


C:\Users\14827\Peptide\ADMET platform\MGA-main\utils\MY_GNN.py:659: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  labels = torch.tensor(labels)


epoch 1/300, training r2+auc -0.1850
epoch 1/300, validation 0.5049, best validation 0.5049 validation result: [0.7823, 0.8375, 0.769, -0.3691]
epoch 2/300, training r2+auc 0.5206
epoch 2/300, validation 0.5838, best validation 0.5838 validation result: [0.8077, 0.8332, 0.7039, -0.0096]
epoch 3/300, training r2+auc 0.5866
epoch 3/300, validation 0.6652, best validation 0.6652 validation result: [0.8261, 0.878, 0.7805, 0.1762]
epoch 4/300, training r2+auc 0.6001
EarlyStopping counter: 1 out of 50
epoch 4/300, validation 0.5618, best validation 0.6652 validation result: [0.8117, 0.8774, 0.7406, -0.1825]
epoch 5/300, training r2+auc 0.5604
EarlyStopping counter: 2 out of 50
epoch 5/300, validation 0.6496, best validation 0.6652 validation result: [0.8386, 0.8844, 0.758, 0.1175]
epoch 6/300, training r2+auc 0.6431
EarlyStopping counter: 3 out of 50
epoch 6/300, validation 0.6600, best validation 0.6652 validation result: [0.8308, 0.9003, 0.7591, 0.1497]
epoch 7/300, training r2+auc 0.6753


EarlyStopping counter: 3 out of 50
epoch 50/300, validation 0.7452, best validation 0.7769 validation result: [0.8913, 0.9357, 0.8451, 0.3086]
epoch 51/300, training r2+auc 0.7987
EarlyStopping counter: 4 out of 50
epoch 51/300, validation 0.7555, best validation 0.7769 validation result: [0.8915, 0.9382, 0.8405, 0.3518]
epoch 52/300, training r2+auc 0.7988
EarlyStopping counter: 5 out of 50
epoch 52/300, validation 0.7619, best validation 0.7769 validation result: [0.8902, 0.9356, 0.8254, 0.3964]
epoch 53/300, training r2+auc 0.8043
EarlyStopping counter: 6 out of 50
epoch 53/300, validation 0.7594, best validation 0.7769 validation result: [0.8951, 0.9327, 0.8347, 0.375]
epoch 54/300, training r2+auc 0.8047
epoch 54/300, validation 0.7780, best validation 0.7780 validation result: [0.8919, 0.9362, 0.8422, 0.4418]
epoch 55/300, training r2+auc 0.8117
EarlyStopping counter: 1 out of 50
epoch 55/300, validation 0.7696, best validation 0.7780 validation result: [0.8888, 0.9357, 0.8372, 0

...

EarlyStopping counter: 2 out of 50
epoch 156/300, validation 0.8021, best validation 0.8022 validation result: [0.9002, 0.9412, 0.8738, 0.4932]
epoch 157/300, training r2+auc 0.8729
EarlyStopping counter: 3 out of 50
epoch 157/300, validation 0.7899, best validation 0.8022 validation result: [0.8975, 0.9443, 0.8707, 0.4469]
epoch 158/300, training r2+auc 0.8723
epoch 158/300, validation 0.8031, best validation 0.8031 validation result: [0.9004, 0.949, 0.8811, 0.4819]
epoch 159/300, training r2+auc 0.8694
EarlyStopping counter: 1 out of 50
epoch 159/300, validation 0.7925, best validation 0.8031 validation result: [0.9012, 0.9426, 0.8718, 0.4543]
epoch 160/300, training r2+auc 0.8784
EarlyStopping counter: 2 out of 50
epoch 160/300, validation 0.7975, best validation 0.8031 validation result: [0.8916, 0.9459, 0.8711, 0.4814]
epoch 161/300, training r2+auc 0.8723
EarlyStopping counter: 3 out of 50
epoch 161/300, validation 0.7933, best validation 0.8031 validation result: [0.8966, 0.9451

epoch 202/300, training r2+auc 0.8837
EarlyStopping counter: 40 out of 50
epoch 202/300, validation 0.7956, best validation 0.8061 validation result: [0.8981, 0.9512, 0.8494, 0.4838]
epoch 203/300, training r2+auc 0.8804
EarlyStopping counter: 41 out of 50
epoch 203/300, validation 0.7885, best validation 0.8061 validation result: [0.8986, 0.9533, 0.8563, 0.4456]
epoch 204/300, training r2+auc 0.8835
EarlyStopping counter: 42 out of 50
epoch 204/300, validation 0.7850, best validation 0.8061 validation result: [0.8961, 0.9488, 0.8656, 0.4293]
epoch 205/300, training r2+auc 0.8832
EarlyStopping counter: 43 out of 50
epoch 205/300, validation 0.7901, best validation 0.8061 validation result: [0.8992, 0.9407, 0.8477, 0.4726]
epoch 206/300, training r2+auc 0.8833
EarlyStopping counter: 44 out of 50
epoch 206/300, validation 0.7916, best validation 0.8061 validation result: [0.9002, 0.945, 0.8584, 0.4627]
epoch 207/300, training r2+auc 0.8866
EarlyStopping counter: 45 out of 50
epoch 207/30

epoch 36/300, training r2+auc 0.7693
EarlyStopping counter: 5 out of 50
epoch 36/300, validation 0.7280, best validation 0.7294 validation result: [0.8771, 0.9302, 0.7576, 0.3469]
epoch 37/300, training r2+auc 0.7714
epoch 37/300, validation 0.7386, best validation 0.7386 validation result: [0.8807, 0.9334, 0.8089, 0.3313]
epoch 38/300, training r2+auc 0.7850
EarlyStopping counter: 1 out of 50
epoch 38/300, validation 0.7332, best validation 0.7386 validation result: [0.8836, 0.9308, 0.8032, 0.3151]
epoch 39/300, training r2+auc 0.7790
EarlyStopping counter: 2 out of 50
epoch 39/300, validation 0.7374, best validation 0.7386 validation result: [0.8851, 0.932, 0.8111, 0.3213]
epoch 40/300, training r2+auc 0.7827
epoch 40/300, validation 0.7558, best validation 0.7558 validation result: [0.8833, 0.9361, 0.8297, 0.3741]
epoch 41/300, training r2+auc 0.7822
EarlyStopping counter: 1 out of 50
epoch 41/300, validation 0.7504, best validation 0.7558 validation result: [0.8781, 0.9365, 0.8094,

epoch 84/300, training r2+auc 0.8384
EarlyStopping counter: 5 out of 50
epoch 84/300, validation 0.7720, best validation 0.7831 validation result: [0.8986, 0.9389, 0.8601, 0.3903]
epoch 85/300, training r2+auc 0.8411
EarlyStopping counter: 6 out of 50
epoch 85/300, validation 0.7766, best validation 0.7831 validation result: [0.8987, 0.938, 0.8355, 0.4341]
epoch 86/300, training r2+auc 0.8389
EarlyStopping counter: 7 out of 50
epoch 86/300, validation 0.7737, best validation 0.7831 validation result: [0.8968, 0.9312, 0.843, 0.4238]
epoch 87/300, training r2+auc 0.8362
EarlyStopping counter: 8 out of 50
epoch 87/300, validation 0.7753, best validation 0.7831 validation result: [0.8949, 0.933, 0.8675, 0.4059]
epoch 88/300, training r2+auc 0.8472
EarlyStopping counter: 9 out of 50
epoch 88/300, validation 0.7756, best validation 0.7831 validation result: [0.8976, 0.939, 0.8636, 0.4024]
epoch 89/300, training r2+auc 0.8463
epoch 89/300, validation 0.7860, best validation 0.7860 validation 

epoch 131/300, training r2+auc 0.8702
EarlyStopping counter: 20 out of 50
epoch 131/300, validation 0.8028, best validation 0.8100 validation result: [0.9024, 0.9441, 0.8684, 0.4963]
epoch 132/300, training r2+auc 0.8676
EarlyStopping counter: 21 out of 50
epoch 132/300, validation 0.8004, best validation 0.8100 validation result: [0.9021, 0.9449, 0.8601, 0.4945]
epoch 133/300, training r2+auc 0.8709
EarlyStopping counter: 22 out of 50
epoch 133/300, validation 0.8017, best validation 0.8100 validation result: [0.9033, 0.9477, 0.8748, 0.4809]
epoch 134/300, training r2+auc 0.8686
EarlyStopping counter: 23 out of 50
epoch 134/300, validation 0.7961, best validation 0.8100 validation result: [0.898, 0.946, 0.8549, 0.4856]
epoch 135/300, training r2+auc 0.8632
EarlyStopping counter: 24 out of 50
epoch 135/300, validation 0.7971, best validation 0.8100 validation result: [0.9029, 0.9471, 0.8626, 0.4759]
epoch 136/300, training r2+auc 0.8667
EarlyStopping counter: 25 out of 50
epoch 136/300